# Importing Libraries

In [ ]:
#Importing Libraries
import pandas as pd  
import datetime
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, kurtosis, skew #correlation coefficient, skewness, and kurtosis 
import seaborn as sns #used for color palletes on graphs and boxplots
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from statsmodels.tsa.seasonal import seasonal_decompose 
from statsmodels.tsa.stattools import adfuller # Dicky Fuller test for stationarity
import statsmodels.api as smm 
import statsmodels.formula.api as sm  # Multiple Linear Regression
from statsmodels.graphics.tsaplots import plot_acf #autocorrelation plot 
from statsmodels.tsa.statespace.sarimax import SARIMAX # SARIMA Model import
from pylab import rcParams # Decomposition of time series

#Another cool model I want to use but can't get to properly install
#from fbprophet import prophet

# Library Data  

## Preprocessing and Merging Visitor + Circulation Variables 

In [ ]:
# Reading in daily circulation data (old method)
#circ_day = pd.read_excel(r"data\KDL Data.xlsx", sheet_name = "phys_circ_day").sort_values(by = "date")
#circ_day['date'] = pd.to_datetime(circ_day['date'], format='%Y-%m') # Converts the date variable into a datetime variable 
#circ_day = circ_day.drop(columns= ["day"]) #Dropping the day variable 

# Reading in new daily circulation data (covers from 2016-2023) 
circ_day = pd.read_excel(r"data\new_daily_checkouts.xlsx").sort_values(by = "date")
circ_day['date'] = pd.to_datetime(circ_day['date'], format='%Y-%m') # Converts the date variable into a datetime variable 
circ_day = circ_day.drop(columns= ["open_hours"]) #Dropping the open hours variable 

# Reading in daily visitor data 
visit_day =  pd.read_excel(r"data\KDL Data.xlsx", sheet_name = "daily_visits").sort_values(by = "date")
visit_day = visit_day[["date", "branch" ,"door_count"]] #Selecting specific variables

#Merging the visitor count and circulation data together by date and branch colums
mergedWV = pd.merge(circ_day, visit_day, on= ['date', 'branch'], how='left')

#Filtering KDL data to only include dates before July 12th to match weather data + filtering out useless branches 
#mergedWV = mergedWV[(mergedWV["date"] >= "8/2/2021") & (mergedWV["date"] <= "6/24/2023") & (~mergedWV["branch"].isin(["BKM", "GFL", "MELCAT", "SC", "GTN"]))]

mergedWV = mergedWV[(mergedWV["date"] >= "8/2/2021") & (~mergedWV["branch"].isin(["BKM", "GFL", "MELCAT", "SC", "GTN"]))]

# Creating a df with only the dates to then fill in the missing dates in the specifc branch dfss 
start_date = datetime(2021, 8, 2)
end_date = datetime(2023, 12, 13)
#end_date = datetime(2023, 6, 24)
date_range = pd.date_range(start=datetime(2021, 8, 2), end=end_date, freq='D')

df_dates = pd.DataFrame({'date': date_range})


#Setting the index to just be the date variable 
mergedWV = mergedWV.set_index('date')


#list of branches I want to specifically look 
branches = ["EGR", "GDV", "CAS"] 
branch_dfs = {}

def expand_dates(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['dayofweek'] = df.index.dayofweek
    #We use the map function along with a lambda function to apply the strftime method to each datetime object in the index. 
    # This approach is necessary because the strftime method is designed to work with individual datetime objects, not with entire Series or DataFrames

    df['dayofweekchar'] = df.index.map(lambda x: x.strftime('%A'))  # Full day name, e.g., "Monday" 
    # Define the desired order of days of the week
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    # Convert 'day_of_week_char' to a categorical variable with the specified order
    df['dayofweekchar'] = pd.Categorical(df['dayofweekchar'], categories=day_order, ordered=True)

    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['month_char'] = df.index.map(lambda x: x.strftime('%b'))  # Full month name, e.g., "September"

    # Define the custom order for the month names
    custom_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    # Convert 'month_char' to a categorical data type with the custom order
    df['month_char'] = pd.Categorical(df['month_char'], categories=custom_order, ordered=True)
     # Sort the DataFrame based on the custom order

    df['year'] = df.index.year

    #turns it not to numeric for whatever reasosn
    df['dayofyear'] = df.index.dayofyear
    df['dayofyear'] = pd.to_numeric(df['dayofyear'])

    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week
    return df


#More efficient to run this function once on the entire dataset then can just use the loop to filter it
mergedWV = expand_dates(mergedWV)

for branch in branches:

    #Assigning each branch to an index in the branch_dfs dictionary
    branch_dfs[branch] = mergedWV[mergedWV["branch"] == branch]

    branch_dfs[branch] =  branch_dfs[branch].drop(columns = ["branch"], axis=1)

    branch_dfs[branch] = pd.merge(df_dates, branch_dfs[branch], on= ['date'], how='left') # Adds all missing dates to each branch df

    branch_dfs[branch]['open'] = 1  # Open by default

    branch_dfs[branch]['open'].loc[branch_dfs[branch]['door_count'].isna()] = 0  # Closed on Sundays in the summer

    branch_dfs[branch] = branch_dfs[branch].set_index('date') #Setting the date variable as the index for all newly created dfs 
    
    globals()[f"{branch}_dataframe"] = branch_dfs[branch] #The globals function allows you to access the variables in the current environment with the f string 

#Cleaning up the envirnment 
del branch, branch_dfs, branches

#circ_day.dtypes

In [ ]:
#Ways of handeling missing values in the branch dfs  

#EGR_dataframe['transactions'].fillna(method='ffill', inplace=True) #Forward Fill method 

EGR_dataframe['transactions'].fillna(0, inplace=True) #Filling in missing values with 0

#EGR_dataframe['transactions'].interpolate(method='linear', inplace=True) #linear interpolation

In [ ]:
# Assuming df is your DataFrame
if EGR_dataframe.index.name == 'date':
    print("The 'date' variable is properly saved as the index.")
else:
    print("The 'date' variable is not the index or not named as 'date'.")

# General EDA 

## Collinearity Checks Among the Predictors 

In [ ]:
# Correlation maxtrix among all the library varaibles 

correlation_matrix = EGR_dataframe.corr(numeric_only = True) 
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Correlation matrix between just on the variables I'm using to measure days/time

correlation_matrix_time = EGR_dataframe[['dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear']].corr()
sns.heatmap(correlation_matrix_time, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
sns.pairplot(EGR_dataframe[['transactions', 'door_count']])
plt.title('Pair Plot')
plt.show()

## Circulation EDA  

In [ ]:
print(EGR_dataframe["transactions"].describe())

In [ ]:
# Density plot (Matplotlib) - Might be cool to add a distriubtion fit like the seaborn pack automatically does

# Calculate kurtosis and skewness
kurtosis_trans = kurtosis(EGR_dataframe["transactions"])
skewness_trans = skew(EGR_dataframe["transactions"])

plt.figure(figsize=(8, 4))
plt.hist(EGR_dataframe["transactions"], density=True, bins=30, alpha=0.5, color='blue')

plt.xlabel('X-axis Label')
plt.ylabel('Density')
plt.title(f'Density Plot\nKurtosis: {kurtosis_trans:.2f}, Skewness: {skewness_trans:.2f}')


plt.text(0.7, 0.9, f'Kurtosis: {kurtosis_trans:.2f}', transform=plt.gca().transAxes, fontsize=12) #The transform=plt.gca().transAxes argument specifies that the text coordinates are given relative to the axes.
plt.text(0.7, 0.80, f'Skewness: {skewness_trans:.2f}', transform=plt.gca().transAxes, fontsize=12)

plt.show()

In [ ]:
#Density plot (Seaborn)

plt.figure(figsize=(8, 4))
sns.kdeplot(EGR_dataframe["transactions"], fill=True, color='green')

plt.xlabel('X-axis Label')
plt.ylabel('Density')
plt.title('Density Plot using Seaborn')

plt.show()

In [ ]:
#Ciruclation by day over time at EGR KDL just checking out the trends and seasonality 

# Plotting the line graph with modified figure size
plt.figure(figsize=(15, 5))
plt.plot(EGR_dataframe.index, EGR_dataframe['transactions'])

# Adding labels and title
plt.xlabel('Date')
plt.ylabel('Transactions')
plt.title('Daily Transactions Since 2021')
plt.grid(True)

# Format x-axis date labels
date_formatter = mdates.DateFormatter("%b '%y")
plt.gca().xaxis.set_major_formatter(date_formatter)

# Set a custom date locator to show every other month
month_locator = mdates.MonthLocator(interval=2)
plt.gca().xaxis.set_major_locator(month_locator)

# Fit a linear regression line (line of best fit)
x = np.arange(len(EGR_dataframe))
y = EGR_dataframe['transactions']
coefficients = np.polyfit(x, y, 4)  # fourth-degree polynomial
trendline = np.poly1d(coefficients)

# Plot the trendline
plt.plot(EGR_dataframe.index, trendline(x), color='red', linestyle='--', label='Trendline')

# Adding a vertical line where we split the dataset at the beginning of 2023
plt.axvline(pd.to_datetime('01-01-2023'), color='black', linestyle='--', label='Vertical Line at x=50')

plt.show()

In [ ]:
#Circulation by day of week (0 is Monday and 6 is Sunday)

fig, ax = plt.subplots(figsize=(10, 8))

sns.boxplot(data=EGR_dataframe, x='dayofweekchar', y='transactions', showmeans = True)

plt.xlabel(None)
plt.ylabel('Daily Transactions')
ax.set_title('Transactions by Day of Week')

plt.show()

In [ ]:
# Circulation by month - also good indicator of seasonality
fig, ax = plt.subplots(figsize=(10, 8))

sns.boxplot(data=EGR_dataframe, x='month_char', y='transactions', palette='Blues', showmeans = True)

ax.set_title('Transactions by Month')
plt.xlabel(None)
plt.ylabel('Daily Transactions')

plt.show()

In [ ]:
# Circulation by quarter 
fig, ax = plt.subplots(figsize=(10, 8))

sns.boxplot(data=EGR_dataframe, x='quarter', y='transactions', palette='Blues', showmeans = True)

ax.set_title('Daily Transactions by Quarter')
plt.xlabel(None)
plt.ylabel('Transactions')

plt.show()

In [ ]:
# Autocorrelation plot

plot_acf(EGR_dataframe['transactions'], lags=30)  # Adjust the number of lags as needed

plt.title('Autocorrelation Plot for Transactions')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')

plt.show()

## Visitor EDA  

In [ ]:
print(EGR_dataframe["door_count"].describe())

In [ ]:
# Density plot (Matplotlib)

# Calculate kurtosis and skewness - CAN'T CALCULATE KURTOSIS AND SKEWNESS HERE BECAUSE OF MISSING VALUES 
kurtosis_visit = kurtosis(EGR_dataframe["door_count"])
skewness_visit = skew(EGR_dataframe["door_count"])

plt.figure(figsize=(8, 4))
plt.hist(EGR_dataframe["door_count"], density=True, bins=30, alpha=0.5, color='blue')
plt.xlabel('visitor count')
plt.ylabel('Density')
plt.title(f'Density Plot\nKurtosis: {kurtosis_visit:.2f}, Skewness: {skewness_visit:.2f}')

plt.text(0.7, 0.9, f'Kurtosis: {kurtosis_visit:.2f}', transform=plt.gca().transAxes, fontsize=12) #The transform=plt.gca().transAxes argument specifies that the text coordinates are given relative to the axes.
plt.text(0.7, 0.80, f'Skewness: {skewness_visit:.2f}', transform=plt.gca().transAxes, fontsize=12)

plt.show()

In [ ]:
# Plotting the line graph with modified figure size
plt.figure(figsize=(15, 5))
plt.plot(EGR_dataframe.index, EGR_dataframe['door_count'])

# Adding labels and title
plt.xlabel('Date')
plt.ylabel('Visitors')
plt.title('Daily Visitors Since 2021')
plt.grid(True)

# Format x-axis date labels
date_formatter = mdates.DateFormatter("%b '%y")
plt.gca().xaxis.set_major_formatter(date_formatter)

# Set a custom date locator to show every other month
month_locator = mdates.MonthLocator(interval=2)
plt.gca().xaxis.set_major_locator(month_locator)

# Fit a linear regression line (line of best fit)
x = np.arange(len(EGR_dataframe))
y = EGR_dataframe['door_count']
coefficients = np.polyfit(x, y, 6)  # fourth-degree polynomial
trendline = np.poly1d(coefficients)

# Plot the trendline
plt.plot(EGR_dataframe.index, trendline(x), color='red', linestyle='--', label='Trendline')

# Adding a vertical line where we split the dataset at the beginning of 2023
plt.axvline(pd.to_datetime('01-01-2023'), color='black', linestyle='--', label='Vertical Line at x=50')

plt.show()

In [ ]:
#Visitors by day of week 

fig, ax = plt.subplots(figsize=(10, 8))

sns.boxplot(data=EGR_dataframe, x='dayofweekchar', y='door_count', showmeans = True)

plt.xlabel(None)
plt.ylabel("Visitors")
ax.set_title('Visitors by Day of Week')

plt.show()

In [ ]:
# Visitors by month 
fig, ax = plt.subplots(figsize=(10, 8))

sns.boxplot(data=EGR_dataframe, x='month_char', y='door_count', palette='Blues', showmeans = True)

plt.xlabel("Month")
plt.ylabel("Visitors")
ax.set_title('Visitors by Month')

plt.show()

In [ ]:
# Autocorrelation plot

plot_acf(EGR_dataframe['door_count'], lags=30)  # Adjust the number of lags as needed

plt.title('Autocorrelation Plot for Door Count')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')

plt.show()

# Preprocessing Weather Datasets

In [ ]:
weather = pd.read_excel(r"data/weather data.xlsx", sheet_name= "weather")

#Creating the a date variable based on the MO, DY, and YEAR columns
weather["date"] =  weather["MO"].astype(str) + "/" + weather["DY"].astype(str) + "/" + weather["YEAR"].astype(str) #concating only works with strings 
weather["date"] = pd.to_datetime(weather["date"], format = "%m/%d/%Y" ) # need to use the pandas to datetime function because the newly created date variable is a pandas object and not a string

#Renaming all the columns to be more readable
weather = weather.rename(columns = {"MO":"Month", "DY":"Day", "YEAR":"Year", "T2M":"temperature", "T2MDEW":"dew_point", "T2M_RANGE":"temp_range", "T2M_MAX":"temp_max", "T2M_MIN":"temp_min", "RH2M": "relative_humidity", "PRECTOTCORR": "precipitation", 
                                    "PS": "pressure", "WS10M": "wind_speed", "WND10M_S": "wind_speed_max", "WS10M_MAX": "wind_max", "WS10M_MIN": "wind_min", "WS10M_RANGE": "wind_range"}) 

#Drop specific variables from our data frame
weather_describe = weather.drop(columns= ["Year", "Month", "Day", "QV2M"])

In [ ]:
# Reading in the second weather dataframe 

weather2 = pd.read_csv(r"data/weather2.csv")

#Renaming all the columns to be more readable
weather2 = weather2.rename(columns = {"PRCP":"Precipitation", "TMAX":"max_temp", "TMIN":"min_temp", "DATE":"Date"}) 

weather2["Date"] = pd.to_datetime(weather2["Date"], format = "%m/%d/%Y")

# Weather EDA

## Summary Statistics for Key Variables

In [ ]:
# Summary statistics for key variables 

# List of important weather variables to plot
y_variables = ["wind_speed", "temperature", "precipitation", "dew_point", "pressure"]

for variable in y_variables:
    print(f'\nSummary Statistics for {variable}:\n{round(weather_describe.describe()[variable], 2)}')

In [ ]:
# List of important weather variables to plot
y_variables = ["wind_speed", "temperature", "precipitation", "dew_point", "pressure"]

# Create subplots to display multiple boxplots
fig, axes = plt.subplots(nrows=len(y_variables), ncols=1, figsize=(8, 5 * len(y_variables)))

# Loop through each variable and create the boxplot
for i, variable in enumerate(y_variables):
    data_to_plot = weather_describe[variable]
    axes[i].boxplot(data_to_plot, vert=True, showmeans= True)  # Use vert=False to create horizontal boxplots

    axes[i].set_xlabel(None)
    axes[i].set_ylabel(variable.title())
    axes[i].set_title(f'Boxplot for {variable.title()}')

    axes[i].grid(True)

# Adjust the layout of subplots and show the plots
plt.tight_layout()
plt.show()

## Distribution for Key Variables

In [ ]:
# List of important weather variables to plot
y_variables = ["wind_speed", "temperature", "precipitation", "dew_point", "pressure"]

# Create subplots to display multiple density plots
fig, axes = plt.subplots(nrows=len(y_variables), figsize=(8, 5*len(y_variables)))

# Loop through each y-variable and create the density plot
for i, variable in enumerate(y_variables):
    var_kurtosis = kurtosis(weather_describe[variable])
    var_skewness = skew(weather_describe[variable])

    axes[i].hist(weather_describe[variable], density=True, bins=30, alpha=0.5, color='blue')

    axes[i].set_xlabel(f'{variable.title()}')
    axes[i].set_ylabel('Density')
    axes[i].set_title(f'Density Plot for {variable.title()} \nKurtosis: {var_kurtosis:.2f}, Skewness: {var_skewness:.2f}')
    axes[i].grid(True)

    axes[i].text(0.7, 0.9, f'Kurtosis: {var_kurtosis:.2f}', transform=axes[i].transAxes, fontsize=12)
    axes[i].text(0.7, 0.80, f'Skewness: {var_skewness:.2f}', transform=axes[i].transAxes, fontsize=12)

# Adjust the layout of subplots and show the plots
#plt.tight_layout()
plt.subplots_adjust(hspace=0.5) # Adjusts the space between subplots
plt.show()

## Plotting Key Weather Varaibles Over Time 

In [ ]:
# Line Graphs 

# List of important weather variables to plot
y_variables = ["wind_speed", "temperature", "precipitation", "dew_point", "pressure"]

# Static X Variable 
x = weather["date"]

# Create separate subplots for each y-variable
num_plots = len(y_variables)
fig, axs = plt.subplots(num_plots, figsize=(10, 5 * num_plots)) #Subplots functions creates a seperate plot for each varaible (not stacking them on same graph)

# Loop through each y-variable and plot it on a separate subplot
for i, y_variable in enumerate(y_variables):
    y = weather[y_variable]
    axs[i].plot(x, y)
    axs[i].set_xlabel("Date")
    axs[i].set_ylabel(y_variable.title())
    axs[i].set_title(f"{y_variable.title()} Over Time")
    axs[i].grid(True)

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
# Boxplots for key variables by month

# List of important weather variables to plot
y_variables = ["wind_speed", "temperature", "precipitation", "dew_point", "pressure"]

# Choosing the X variable for how I want to group the data
x = 'Month'   # Year Month Day

# Create subplots to display multiple boxplots
fig, axes = plt.subplots(nrows=len(y_variables), ncols=1, figsize=(10, 5 * len(y_variables)))

# Loop through each variable and create the boxplot
for i, variable in enumerate(y_variables):
    sns.boxplot(data=weather, x=x, y=variable, palette='Blues', showmeans=True, ax=axes[i])

    axes[i].set_xlabel(None)
    axes[i].set_ylabel(variable.title())
    axes[i].set_title(f'Boxplot for {variable.title()}')

    axes[i].grid(True)

# Adjust the layout of subplots and show the plots
#plt.tight_layout()
plt.subplots_adjust(hspace=0.4)
plt.show()

# Megring the Weather and Circulation Datasets Together Based on Date 

In [ ]:
EGR_all = pd.merge(EGR_dataframe, weather, on='date', how='left')

In [ ]:
EGR_all = EGR_all.set_index('date')

## Bivariate anaylsis: Weather Data vs. Circulation/Visitor Counts

In [ ]:
# Assuming you have already defined EGR_all and x here

# Define X variable 
x = "transactions" # door_count transactions
x_var = EGR_all["transactions"] 

# List of key weather variables to iterate through against our x variable
y_variables = ["wind_speed", "temperature", "precipitation", "dew_point", "pressure", 'door_count']

# Create subplots to display multiple plots
fig, axes = plt.subplots(nrows=len(y_variables), ncols=1, figsize=(8, 5*len(y_variables)))

# Loop through each y variable and create the scatter plot
for i, variable in enumerate(y_variables):
    y = EGR_all[variable]

    # Calculate the line of best fit using numpy's polyfit function
    coefficients = np.polyfit(x_var, y, 1)
    polynomial = np.poly1d(coefficients)

    # Calculate Pearson's correlation coefficient (R) and its p-value
    correlation_coefficient, p_value = pearsonr(x_var, y)

    # Create the scatter plot
    axes[i].scatter(x_var, y, label='Data Points')

    # Add the line of best fit to the plot
    axes[i].plot(x_var, polynomial(x_var), color='red', label='Line of Best Fit')

    # Add the correlation coefficient to the plot
    axes[i].text(1375, np.mean(y), f"Pearson's R: {correlation_coefficient:.2f}", fontsize=12)

    # Add labels and legend
    axes[i].set_xlabel(f'{x.title()}')
    axes[i].set_ylabel(variable.title())
    axes[i].set_title(f'Scatter Plot Between {x.title()} and {variable.title()}')
    axes[i].grid(True)
    axes[i].legend()

# Adjust the layout of subplots and show the plots
plt.tight_layout() # Adjusts the spacing between subplots to make sure they don't overlap 
plt.show()

# Multiple Linear Regression 

In [ ]:
#### I could make this model even better potentially by adding in data for all the missing value days ####

In [ ]:
# Splitting the dataset into a training and testing set
trainMLR = EGR_all.loc[EGR_all.index < '01-01-2023']
testMLR = EGR_all.loc[EGR_all.index >= '01-01-2023']

In [ ]:
import statsmodels.formula.api as sm

EGR_all.columns

# Create a linear regression model
MLR_model = sm.ols('transactions ~ door_count + dayofweek + quarter + month + year + dayofyear + dayofmonth + dayofyear + precipitation + temp_max  + relative_humidity' , data=trainMLR).fit()

# Print the summary of the model
print(MLR_model.summary())

############ Potentially try to insitutute forwards, backward, or stepwise elimination??? ###############

## Assumption Checking 

### Form of Model Assumption

In [ ]:
# Get the predicted values and residuals
predicted_values = MLR_model.predict()
residuals = MLR_model.resid

#Residual vs predicted plot 
# Checking Linearity, mean zero, constant variance

plt.scatter(predicted_values, residuals)

plt.title('Residuals vs. Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')

plt.axhline(0, color='red', linestyle='--')

plt.show()

In [ ]:
# Residual vs Fitted 

# fitted values
model_fitted_y = MLR_model.fittedvalues

#  Plot
plot = sns.residplot(x=model_fitted_y, y = predicted_values, data=EGR_all, lowess=True, 
                     scatter_kws={'alpha': 0.5}, 
                     line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

# Titel and labels
plot.set_title('Residuals vs Fitted')
plot.set_xlabel('Fitted values')
plot.set_ylabel('Residuals')

### Error Assumption  

In [ ]:
# 2. Independence of Errors Assumption (Durbin-Watson Test)
# Durbin-Watson test checks for autocorrelation in residuals (values between 0 and 4, with 2 indicating no autocorrelation)
durbin_watson_statistic = sm.stats.stattools.durbin_watson(residuals)
print(f"Durbin-Watson Statistic: {durbin_watson_statistic}")

In [ ]:
# Check for normality of residuals using a Q-Q plot

sm.qqplot(residuals, line='s')
plt.title('Q-Q Plot of Residuals')
plt.show()

In [ ]:
# 6. Check for constant variance in residuals (Goldfeld-Quandt Test)
# This test checks if the variance of residuals is constant across different subsets of the data
# It is often used when you suspect heteroscedasticity
from statsmodels.stats.diagnostic import het_goldfeldquandt
f_stat, p_value, _ = het_goldfeldquandt(residuals, model.model.exog)
print(f"Goldfeld-Quandt Test p-value: {p_value}")

### Influence Diagnostics (Outliers and Influential Points) 

In [ ]:
#Check for outliers (e.g., Cook's distance, leverage)
# Identify and investigate potential outliers using Cook's distance or leverage plots

#create instance of influence
influence = MLR_model.get_influence()

#obtain Cook's distance for each observation
cooks = influence.cooks_distance

plt.scatter(EGR_all.index, cooks[0])
plt.xlabel('Observation')
plt.ylabel('Cooks Distance')

#Adding the index number for each point to investigate further 
for i, (x, y) in enumerate(zip(EGR_all.index, cooks[0])):
    plt.text(x, y, str(i), fontsize=12, ha='center', va='bottom')

plt.show()

In [ ]:
# 8. Check for influential points (e.g., DFBETAS, DFFITS)

## Predictions

In [ ]:
# Adding the predicted values of our model back to our original df 

testMLR['predicted'] = MLR_model.predict(testMLR)

In [ ]:
# Actual vs Predicted graph

#plt.plot(trainSAR, label='Train') #training data
plt.plot(testMLR['transactions'], label='Train') #testing data we want to predict
plt.plot(testMLR['predicted'], label='Prediction') #data our model predicted for the taining set 
plt.show()

In [ ]:
# verifying how well the model performed 

boxplote = sns.boxplot(x=testMLR.index.dayofweek, y=testMLR["error"])
plt.xlabel("Day of the Week")
plt.ylabel("Error")
plt.xticks(range(7), ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
plt.title("Error Distribution by Day of the Week")

# Adding minor ticks on the y-axis
plt.grid(axis='y', which='both', linestyle='dotted')

plt.show()

In [ ]:
#Confidence Intervals (95% is the default)

confidence_intervals = model.conf_int()
print(confidence_intervals)

# Splitting the Dataset into Training and Test Sets 

In [ ]:
#splitting the dataset into training (prior to 2023) and testing sets (after 2023)

train = EGR_dataframe.loc[EGR_dataframe.index < '01-01-2023']
test = EGR_dataframe.loc[EGR_dataframe.index >= '01-01-2023']

In [ ]:
#Creating the model 

FEATURES = ['dayofyear', 'dayofweek', 'quarter', 'month', 'year', 'open']
TARGET = 'transactions'

x_train = train[FEATURES]
y_train = train[TARGET]

x_test = test[FEATURES]
y_test = test[TARGET]

# XGBRegressor Model 

In [ ]:
#Hyperparameter Grid Search

from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    # Add more hyperparameters to tune as needed
}

# Create the XGBoostRegressor model
xgb_model = xgb.XGBRegressor()

# Initialize the GridSearchCV object with the model, hyperparameter grid, and scoring metric
grid_search = GridSearchCV(xgb_model, param_grid, scoring='r2', cv=5)

# Fit the GridSearchCV to find the best hyperparameters
grid_search.fit(x_train, y_train)

# Get the best hyperparameters and corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)

In [ ]:
#325 appears to be the sweet spot before overfitting 
XGBoost_reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                       n_estimators=300, # Number of boosting rounds orig number:325
                       early_stopping_rounds=50,
                       objective='reg:linear',
                       max_depth=5, # Maximum depth of each tree
                       learning_rate=0.01) # Learning rate orig number: 0.01

#The probelm with the 'oh' error is in the fit code                        
XGBoost_reg.fit(x_train, y_train,
        eval_set=[(x_train, y_train), (x_test, y_test)],
        verbose=25)

## Evaluating the XGBRegressor Model with Feature Importance Chart, Correlation Matrix, and Residual Mean Square Error 

In [ ]:
#Feature Importance Graph

fi = pd.DataFrame(data=XGBoost_reg.feature_importances_,
             index=XGBoost_reg.feature_names_in_,
             columns=['importance'])
fi.sort_values('importance').plot(kind='barh', title='Feature Importance')
plt.show()

In [ ]:
#Correlation matrix

# Calculate the correlation matrix
correlation_matrix = EGR_dataframe.corr()

# Print the correlation matrix
print(np.round(correlation_matrix, decimals=2))

# Create correlation matrix plot (heatmap)
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
#Calculating residual mean square error 

score = np.sqrt(mean_squared_error(test['transactions'], XGBoost_reg.predict(x_test)))
print(f'RMSE Score on Test set: {score:0.2f}')

## Looking at the errors between |actual-predicted| 

In [ ]:
# Data wrangling to get the predicted values for our xgb regressor model into our original 'EGR dataframes' dataset

#This is so dumb to avoid copy error BS 
test = test.copy()

# Getting the predicted values from our xgb regressor model as a np array 
test["predicted"] = np.round(XGBoost_reg.predict(x_test).astype(float), decimals = 2) #need to use np.round because reg.predict(X_test) returns a numpy array and we need to convert "predicted" to a float 64 intead of a float 32

# Merging the predicted variable from our test dataframe to into the oringal 'EGR dataframes' dataset (we can't just input the 'predicted array directly in this dataset because the oringal dataset has both training and test set)
EGR_dataframe = EGR_dataframe.merge(test[["predicted"]], how = "left", left_index = True, right_index = True)

#Calculating the error between the predicted and actual values
EGR_dataframe["error"] = abs(EGR_dataframe["transactions"] - EGR_dataframe["predicted"])

In [ ]:
# plotting predicted vs actual cirulation data over time

ax = EGR_dataframe.loc[(EGR_dataframe.index >= '01-01-2023')]['transactions'] \
    .plot(figsize=(15, 5), title='Week Of Data')

EGR_dataframe.loc[(EGR_dataframe.index > '01-01-2023')]['predicted'] \
    .plot()

plt.legend(['True Data','Prediction'])
plt.show()

In [ ]:
#Grouping the error variable by day of the week and seeing if we have trouble predicting particular days 

boxplote = sns.boxplot(x=EGR_dataframe.index.dayofweek, y=EGR_dataframe["error"])
plt.xlabel("Day of the Week")
plt.ylabel("Error")
plt.xticks(range(7), ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
plt.title("Error Distribution by Day of the Week")

# Adding minor ticks on the y-axis
plt.grid(axis='y', which='both', linestyle='dotted')

plt.show()

In [ ]:
#In this example, I've created two grouped_data variables: grouped_data_mean and grouped_data_median. Each variable represents the mean and median error values grouped by the day of the week.
#The bar chart is created using plt.bar(). By specifying different x-coordinates for the mean and median bars (index and index + bar_width), the bars are displayed side by side. The legend is used to differentiate between the mean and median error bars.
#To display the values on top of each bar, I've used the annotate() function to add text annotations. The height of each bar is extracted using rect.get_height() and displayed with two decimal places.

# Grouping the error variable by the day of the week
grouped_data_mean = EGR_dataframe.groupby(EGR_dataframe.index.dayofweek)["error"].mean()
grouped_data_median = EGR_dataframe.groupby(EGR_dataframe.index.dayofweek)["error"].median()

# Creating a bar chart
bar_width = 0.35
index = np.arange(7)

fig, ax = plt.subplots()
rects1 = ax.bar(index, grouped_data_mean, bar_width, label='Mean Error')
rects2 = ax.bar(index + bar_width, grouped_data_median, bar_width, label='Median Error')

# X-axis labels and ticks
plt.xlabel("Day of the Week")
plt.ylabel("Error")
plt.xticks(index + bar_width/2, ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
plt.title("Mean and Median Error by Day of the Week")
plt.legend()

# Displaying the values on top of the bars
for rect in rects1+rects2:
    height = rect.get_height()
    ax.annotate(f'{height:.2f}', xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom')

plt.show()

In [ ]:
# Clearing all variables from global namespace
globals().clear()

# ARIMA, SARIMA, and SARIMAX Models

In [ ]:
#Check for stationarity - need to figure out how to do this - Stationarity allows us to assume past behavior will predict future behavior 

## Data Cleaning, checks, and Fixes 

In [ ]:
# Splitting the data set appropriately into training and testing set

trainSAR = EGR_dataframe.loc[EGR_dataframe.index < '01-01-2023']
trainSAR = trainSAR[["transactions", "open"]] # Keeps just the transactions and date columns in the df (don't need to specify date because it's the index)

testSAR = EGR_dataframe.loc[EGR_dataframe.index >= '01-01-2023']
testSAR = testSAR[["transactions", "open"]]

In [ ]:
# Two ways to check for stationarity: 1.Rolling statistics 2. Augmented Dickey-Fuller test

rolling_mean = EGR_dataframe["transactions"].rolling(window=7).mean() 
rolling_sd =  EGR_dataframe["transactions"].rolling(window=7).std() 

# Plotting the line graph with modified figure size
plt.figure(figsize=(15, 5))
plt.plot(EGR_dataframe.index, EGR_dataframe['transactions'])
plt.plot(rolling_mean, color = 'red', label = 'Rolling Mean')
plt.plot(rolling_sd, color = 'black', label = 'Rolling Standard Deviation')

# Adding labels and title
plt.xlabel('Date')
plt.ylabel('Transactions')
plt.title('Line Graph: Transactions')
plt.grid(True)

# Format the x-axis dates in the format "Jan '23"
date_format = mdates.DateFormatter("%b '%y")
plt.gca().xaxis.set_major_formatter(date_format)
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))

# Rotate the date labels for better visibility
plt.xticks(rotation=30)
plt.legend()

# Adding a vertical line where we split the dataset 
plt.axvline(pd.to_datetime('01-01-2023'), color='black', linestyle='--', label='Vertical Line at x=50')

plt.show()

In [ ]:
rolling_mean = EGR_dataframe["transactions"].rolling(window=7).mean() 
rolling_sd =  EGR_dataframe["transactions"].rolling(window=7).std() 

baskets = np.sqrt(EGR_dataframe["transactions"])

plt.plot(baskets)

In [ ]:
# Dickey-Fuller test

result = adfuller(EGR_all['transactions'])
print('ADF Statistic: {}'.format(result[0]))
print('p-value: {}'.format(result[1]))
print('Critical Values:')
for key, vlaue in result[4].items():
    print('\t{}: {}'.format(key, vlaue))

In [ ]:
#seasonal decomposition
results = seasonal_decompose(EGR_dataframe["transactions"], period=50) #The period parameter will attempt to identify a repeating pattern or seasonality with a length of 111 data points in the time series.
results.plot(); 

In [ ]:
#Plotting the training and testing data side by side 

trainSAR['transactions'].plot()
testSAR['transactions'].plot()

## Auto Arima Model

In [ ]:
from pmdarima import auto_arima

model = auto_arima(trainSAR['transactions'], trace=True, m = 6, suppress_warnings=True, seasonal=True, stepwise=True) # I don't need to specify the transactions variable here since that's the only 
model.fit(trainSAR['transactions'])                                                                                                                           # but it helps with clarity of code helps with clarity of code other variable in my df                                                                                                   

In [ ]:
predictions_auto = pd.Series(model.predict(n_periods=len(testSAR))) # with n_periods we predict N steps into the future

predictions_auto.index = testSAR.index

In [ ]:
# plot the predictions for validation set for the auto arima

#plt.plot(trainSAR, label='Train') #training data
plt.plot(testSAR["transactions"], label='Train') #testing data we want to predict
plt.plot(predictions_auto, label='Prediction') #actual predictions 
plt.axhline(800, color='black', linestyle='--') #data our model predicted for the taining set 
plt.show()

## Arima Model

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# ARIMA (AutoRegressive Integrated Moving Average) model with order (p, d, q)

# Create and fit the ARIMA model on the training set
arima_model = ARIMA(trainSAR['transactions'], order=(4, 0, 0), freq='D')
fitted_arima_model = arima_model.fit()

# Make predictions on the test set
predictions_arima = pd.Series(fitted_arima_model.predict(start=len(trainSAR), end=len(trainSAR) + len(testSAR) - 1))

predictions_arima.index = testSAR.index

In [ ]:
# Graphing actuals vs predicitions

plt.plot(testSAR["transactions"], label='Train')  # Testing data we want to predict
plt.plot(predictions_arima, label='Prediction')  # Actual predictions
plt.axhline(800, color='black', linestyle='--')  # Data our model predicted for the training set

plt.title('ARIMA Model Predictions on Test Set')

plt.show()

## SARIMA Model

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Example SARIMA (Seasonal ARIMA) model with seasonal_order (P, D, Q, S)

sarima_model = SARIMAX(trainSAR['transactions'], order=(2,0,3), seasonal_order=(2,0,1, 6), freq='D')  # Adjust orders and seasonal_order as needed
fitted_sarima_model = sarima_model.fit()
predictions_sarima = fitted_sarima_model.predict(start=len(trainSAR), end=len(trainSAR) + len(testSAR) - 1)

predictions_sarima.index = testSAR.index

In [ ]:
# Graphing actuals vs predicitions

plt.plot(testSAR["transactions"], label='Train')  # Testing data we want to predict
plt.plot(predictions_sarima, label='Prediction')  # Actual predictions
plt.axhline(800, color='black', linestyle='--')  # Data our model predicted for the training set

plt.title('SARIMA Model Predictions on Test Set')

plt.show()

## SARIMAX Model

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# SARIMAX - Seasonal ARIMA with exogenous variables - that influence/explain other varaibles in the model

sarimax_model = SARIMAX(trainSAR['transactions'], exog=trainSAR[['open']], order=(2,0,3), seasonal_order=(2,0,1, 6), freq='D')
fitted_sarimax_model = sarimax_model.fit()
predictions_sarimax = fitted_sarimax_model.predict(start=len(trainSAR), end=len(trainSAR) + len(testSAR) - 1, freq='D', exog=testSAR[['open']], dynamic=False)

predictions_sarimax.index = testSAR.index

In [ ]:
#Graphing SARIMA with exogenous variables

plt.plot(testSAR["transactions"], label='Train')  # Testing data we want to predict
plt.plot(predictions_sarimax, label='Prediction')  # Actual predictions
plt.axhline(800, color='black', linestyle='--')  # Data our model predicted for the training set

plt.title('SARIMAX Model Predictions on Test Set')

plt.show()

## Model evaluations 

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate_forecast(y,pred):
    results = pd.DataFrame({'r2_score':r2_score(y, pred),
                           }, index=[0])
    results['mean_absolute_error'] = mean_absolute_error(y, pred)
    results['median_absolute_error'] = median_absolute_error(y, pred)
    results['mse'] = mean_squared_error(y, pred)
    #results['msle'] = mean_squared_log_error(y, pred)
    results['mape'] = mean_absolute_percentage_error(y, pred)
    results['rmse'] = np.sqrt(results['mse'])
    return results

# predictions_sarimax, predictions_sarima, predictions_arima, predictions_auto

evaluate_forecast(testSAR['transactions'], predictions_sarimax)

In [ ]:
pred_ci = pred.conf_int()
ax = y['1949':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='Forecast', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Passengers')
plt.legend()
plt.show()

In [ ]:
# Assuming df is your DataFrame
if trainSAR.index.name == 'date':
    print("The 'date' variable is properly saved as the index.")
else:
    print("The 'date' variable is not the index or not named as 'date'.")

In [ ]:
#Could also finsih this assessment off by using this model to acutally predict future values and train the model on the entire dataset

# Fitting Long Short-Term Memory (LSTM) Neural Network 

In [ ]:
# MIGHT BE KINDA COOL TO FEED A MODEL LIKE THIS ALL BRANCHES CHECKOUTS, POTENTIALLY THE MORE DATA WILL 

In [ ]:
mergedWV

cheese = mergedWV.groupby("date")["transactions"].sum()

cheese

In [ ]:
#import tensorflow as tf
import os

In [ ]:
#Reformating the data - each variable is bracketed in a list in case we want to add more variables  

# Numpy arrays must all be the same data type, numpy makes it easy to perform mathmatical operations on entire arrays

# [[[1], [2], [3], [4], [5]]] [6]   
# [[[2], [3], [4], [5], [6]]] [7]
# [[[3], [4], [5], [6], [7]]] [8]

def df_to_X_y(df, window_size=5): # window size is the last X amount of measurements we're looking at to the predict the next value 
  df_as_np = df.to_numpy()  # Convertning df to numpy

  X = [] # the 5 input numbers as a part of our window size used to predict y
  y = [] # the value we predict

  for i in range(len(df_as_np)-window_size): 
    row = [[a] for a in df_as_np[i:i+window_size]] # Creating our row which will store each value within our window size and then wrapping each of those values into a list with the "[a] for a in" part
    X.append(row) 
    label = df_as_np[i+window_size]
    y.append(label)
  return np.array(X), np.array(y)

In [ ]:
# calling the above functions

transactions = EGR_dataframe['transactions']

WINDOW_SIZE = 5
X1, y1 = df_to_X_y(transactions, WINDOW_SIZE)
X1.shape, y1.shape

In [ ]:
X_train1, y_train1 = X1[:550], y1[:550]
X_val1, y_val1 = X1[550:600], y1[550:600]
X_test1, y_test1 = X1[600:], y1[600:]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam


model1 = Sequential()
model1.add(InputLayer((5,1)))
model1.add(LSTM(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'relu'))

model1.summary()

In [ ]:
cp = ModelCheckpoint('model1/', save_best_only=True) # only save the best model defined by the one that has the lowest validation loss
model1.compile(loss= MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()]) #

In [ ]:
model1.fit(X_train1, y_train1, validation_batch_size=(X_val1, y_val1), epochs=10, callbacks=[cp])

In [ ]:
from tensorflow.keras.models import load_model
model1 = load_model('model1/')

In [ ]:
train_predictions = model1.predict(X_train1).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train1})
train_results

In [ ]:
#Seeing how well the model did on the training set 
plt.plot(train_results['Train Predictions'][50:100])
plt.plot(train_results['Actuals'][50:100])

In [ ]:
val_predictions = model1.predict(X_val1).flatten()
val_results = pd.DataFrame(data={'Val Predictions':val_predictions, 'Actuals':y_val1})
val_results

In [ ]:
#Seeing how well our model did on our validation data

plt.plot(val_results['Val Predictions'][:100])
plt.plot(val_results['Actuals'][:100])

In [ ]:
#Seeing how well our model did on our test data (data that has never been before by the model)

test_predictions = model1.predict(X_test1).flatten()
test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':y_test1})
test_results

In [ ]:
plt.plot(test_results['Test Predictions'][:100])
plt.plot(test_results['Actuals'][:100])

# Extra Code

In [ ]:
#Ideas for creating a forecasting model that didn't pan out 

#The following models are not really good for time series forecasting 

#from sklearn import svm
#from sklearn.linear_model import Perceptron
#from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import GaussianNB
#from sklearn.neighbors import KNeighborsClassifier

#Other idea of trying to map weekdays to a specific number to then input into the model 

#If we use one-hot encoding like below effectively, we decide that the order of days no longer matters. Is it the case? Not always. Usually, we should not use one-hot encoding to encode days of weeks.
    #branch_dfs[branch] = pd.get_dummies(branch_dfs[branch], columns = ["day"]) #Converts the categorical variables into dummy variables

# Define a dictionary mapping weekdays to numerical values
#weekday_mapping = {
   # "Monday": 1,
 #   "Tuesday": 2,
 #   "Wednesday": 3,
 #   "Thursday": 4,
 #   "Friday": 5,
  #  "Saturday": 6,
  #  "Sunday": 7}

#branch_dfs[branch].loc[:, "n_day"] = branch_dfs[branch]["day"].map(weekday_mapping)

#angular distance method for input week days into a model (saves the order of the days)
# https://www.mikulskibartosz.name/time-in-machine-learning/

#circ_day['day_of_week_sin'] = np.sin(circ_day['day'] * (2 * np.pi / 7))
#circ_day['day_of_week_cos'] = np.cos(circ_day['day'] * (2 * np.pi / 7))


#Splitting the dataset into training and test sets

#X_train, X_test, y_train, y_test = train_test_split(EGR_dataframe.drop('transactions', axis=1), EGR_dataframe['transactions'], test_size=0.2, random_state=42)

# The df.drop('target_column', axis=1) selects all columns except the target column as the input features (X)
#The test_size parameter specifies the proportion of the data to be allocated to the test set. In the example, 20% of the data is reserved for testing, indicated by test_size=0.2. You can adjust this 
# value according to your needs.
#The random_state parameter sets the seed for the random number generator, ensuring reproducibility of the split. You can change the value of random_state to get different random splits or omit it altogether 
#to have different splits on each run.